In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_palette("colorblind")

# Personalized Fashion Recommendations

## Objectives:
- To predict future potential customers purchasing
- Propose outfit combinations

## Dataset
https://www.kaggle.com/competitions/h-and-m-personalized-fashion-recommendations/data

## Metric

Comparing the predicted purchases with the purchases done in the 7 days after the time of training set period.

## Naive/Baseline model

- **Naive model**: All customers buy any random article.
- **Baseline model**: The customer purchases the same articles purchased in the past.

## Reading Data

In [2]:
# df_customers = pd.read_csv("../fashion_dataset/customers.csv.zip")
# df_transactions = pd.read_csv("../fashion_dataset/transactions_train.csv.zip")

In [3]:
df_articles = pd.read_csv("../fashion_dataset/articles.csv.zip")

In [4]:
df_articles.shape

(105542, 25)

In [5]:
df_articles.sample(2)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
94164,861036002,861036,Spritz top,258,Blouse,Garment Upper body,1010016,Solid,8,Dark Grey,...,Blouse & Dress,A,Ladieswear,1,Ladieswear,18,Womens Trend,1010,Blouses,Blouse in a crisp weave with gathered seams at...
27373,619878001,619878,Blueberry dress,265,Dress,Garment Full body,1010001,All over pattern,23,Dark Yellow,...,Blouse & Dress,A,Ladieswear,1,Ladieswear,18,Womens Trend,1010,Blouses,NaN
